In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
BATCH_SIZE = 100

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./cifar_subset', train=True, download=False, transform=transform, force_integrity=True)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

test_data = torchvision.datasets.CIFAR10(root='./cifar_subset', train=False, download=False, transform=transform, force_integrity=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

In [5]:
# Get CIFAR SUBSET

In [6]:
# import pickle
# with open('./data/cifar-10-batches-py/batches.meta','rb') as F:
#     bd = pickle.load(F, encoding='latin1')
# bd_sub_meta=b
# bd_sub_meta['num_cases_per_batch'] = 974
# bd_sub_meta['label_names'] = ['automobile']
# ['num_vis'] = 3072

In [7]:
# import pickle
# with open('./data/cifar-10-batches-py/test_batch','rb') as F:
#     bd = pickle.load(F, encoding='latin1')


# picked_category = 1

# batch_labels = np.array(bd['labels'])
# batch_samples = np.array(bd['data'])
# batch_filenames = bd['filenames']
# chosen_ids = batch_labels == picked_category

# chosen_samples = batch_samples[chosen_ids]

# chosen_labels = batch_labels[chosen_ids]

# for i in range(len(chosen_ids)):
#     if chosen_ids[i]:
#         chosen_filenames.append(batch_filenames[i])

# bd_subset = {}
# bd_subset['batch_label'] = "Category 1 subset from Training batch 1"

# bd_subset['labels'] = batch_labels
# bd_subset['data'] = chosen_samples
# bd_subset['labels'] = chosen_labels
# bd_subset['filenames'] = chosen_filenames
# with open('./cifar_subset/cifar-10-batches-py/test_batch','wb') as F:
#     pickle.dump(bd, F)

In [33]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [34]:
#Now using the AlexNet
AlexNet_Model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)
AlexNet_Model.eval();

Using cache found in /home/smadan/.cache/torch/hub/pytorch_vision_v0.6.0


In [35]:
import torch.nn as nn
AlexNet_Model.classifier[1] = nn.Linear(9216,4096)
AlexNet_Model.classifier[4] = nn.Linear(4096,1024)
AlexNet_Model.classifier[6] = nn.Linear(1024,2)

In [36]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [37]:
AlexNet_Model.to(device);

In [38]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(AlexNet_Model.parameters(), lr=0.001, momentum=0.9)

In [39]:
from utils import MLP
from utils import CMA_info
import utils

In [40]:
def get_uniform_limits(mean,std):
    A = 3.45*std #b-a
    B = mean*2 #a+b
    b = (B+A)/2
    a = b - A
    return a,b

In [41]:
N_dim = 100


x1_min = 0.06075
x1_max = 0.83925

x2_min = 20
x2_max = 40

test_min = -1
test_max= 1

limits = [x1_min, x1_max, x2_min, x2_max]
test_limits = [test_min, test_max]

num_samples = 50

In [42]:
limits_1 = get_uniform_limits(0.2,0.225)
limits_2 = get_uniform_limits(0.7,0.225)

In [43]:
limits = limits_1 + limits_2

In [47]:
from tqdm.notebook import tqdm

In [48]:
import time

In [55]:
labels

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1], device='cuda:0')

In [61]:
for epoch in range(100):  # loop over the dataset multiple times
    corrects = 0
    totals = 0
    
    start_time = time.time()
    for data in tqdm(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        noise_dset = utils.make_image_samples([int(len(inputs)/2),3,224,224], limits)
        noisy_inputs = inputs + noise_dset[0].to(device)
        noisy_labels = noise_dset[1].long().to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = AlexNet_Model(noisy_inputs)
        
        preds = torch.argmax(output,dim=1)
        corrects += torch.sum(noisy_labels == preds)
        totals += len(noisy_labels)
        
        loss = criterion(output, noisy_labels)
        loss.backward()
        optimizer.step()

        #Time
        end_time = time.time()
        time_taken = end_time - start_time
    print("Epoch %s Accuracy:%s"%(epoch, corrects/totals))

print('Finished Training of AlexNet')

  0%|          | 0/49 [00:00<?, ?it/s]

Epoch 0 Accuracy:tensor(0.9854, device='cuda:0')


  0%|          | 0/49 [00:00<?, ?it/s]

Epoch 1 Accuracy:tensor(0.9899, device='cuda:0')


  0%|          | 0/49 [00:00<?, ?it/s]

...

Epoch 78 Accuracy:tensor(1., device='cuda:0')


  0%|          | 0/49 [00:00<?, ?it/s]

Epoch 79 Accuracy:tensor(1., device='cuda:0')


  0%|          | 0/49 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [62]:
noise_classes = ['0','1']

In [66]:
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
with torch.no_grad():
    for data in tqdm(testloader):
        images, labels = data[0].to(device), data[1].to(device)
        
        noise_dset = utils.make_image_samples([int(len(images)/2),3,224,224], limits)

        images = images + noise_dset[0].to(device)
        labels = noise_dset[1].long().to(device)
        
        outputs = AlexNet_Model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('Accuracy of %s : %2d %%' % (
        noise_classes[i], 100 * class_correct[i] / class_total[i]))

  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy of 0 : 100 %
Accuracy of 1 : 100 %


In [67]:
class_correct

[487.0, 487.0]

In [68]:
#Verifying average accuracy of the network
avg = 0
for i in range(2):
  temp = (100 * class_correct[i] / class_total[i])
  avg = avg + temp
avg = avg/2
print('Average accuracy = ', avg)

Average accuracy =  100.0


In [69]:
torch.save(AlexNet_Model, 'alexnet_noise_on_cars.pt')